# Class Scheduling Problem as a Constraint Satisfaction Problem (CSP) - Group 08
**This iPY notebook describes the implementation of a Class Scheduling Problem as a Constraint Satisfaction Problem (CSP), made by Group 08 (Augusto Pereira - 21136, Duarte Melo - 21149 and Nuno Veloso - 10411)**    
This implementation uses the AIMA (Artificial Intelligence: A Modern Approach) code, available on GitHub at https://github.com/aimacode/aima-python  


The group used the following files from AIMA code:
- csp.py [especificar funçoes]
- utils.py [especificar funçoes]
- search.py [especificar funçoes]

In [ ]:
from lessons import Lesson
from csp import *
from my_utils import *
import time

# Problem formulation
The goal of this agent is to, after receiving as input a set of data relative to classes, subjects and rooms, return a schedule that satisfies all the constraints specified.  
Some of the constraints are specified in the Project Guide, but the group decided to add some more to turn the problem more realistic.

## Input Data
The data received as input, used by the algorithm, is specified in three dictionaries.

In [ ]:
# classes used in the algorithm, the name is irrelevant since only the numbers will be used
classes = {
    1: "LESI",
    2: "LESI-PL"
}

# subjects used in the algorithm, the algorithm accepts, at the moment, 5 subjects, the name is irrelevant
subjects =  {
    1: "Mobile",
    2: "Programming",
    3: "Artificial Intelligence",
    4: "Data Structures",
    5: "APIs"
}

# rooms used in the algorithm, the name is irrelevant but ROOM 0 = online!
rooms = {
    0: "Online",
    1: "Sala L",
    2: "Sala T",
    3: "Sala N"
}

In the end of the execution, the algorithm will return a schedule using these dictionaries keys.



## Variables
The group created several variables to be able to create constraints and return a solution.
  
The "main" variable is the Lesson, but each Lesson has some kind of "sub-variables".

Each class has 10 Lessons, one "free day" (a day without lessons) and a "favourite room" (a room where the class has 2 to 4 lessons).  
Each Lesson has these sub-variables (in this case, Lesson nr. 0):  
- L0.c -> class that attends Lesson 0
- L0.su -> subject related to Lesson 0
- L0.d -> duration of Lesson 0
- L0.w -> weekday of Lesson 0
- L0.st -> start time of Lesson 0
- L0.r -> the room where Lesson 0 is attended

This is a bit of a result given by the script main.py, to explain how the result uses variables:
```

```{'L0.fd': 5, 'L0.fr': 1, 'L0.c': 1, 'L1.c': 1, 'L2.c': 1, 'L3.c': 1, 'L4.c': 1, 'L5.c': 1, 'L6.c': 1, 'L7.c': 1, 'L8.c': 1, 'L9.c': 1, 'L0.su': 3, 'L0.d': 2, 'L0.w': 4, 'L0.st': 8, 'L0.r': 1, 'L1.su': 3, 'L1.d': 2, 'L1.w': 2, 'L1.st': 10, 'L1.r': 2, 'L2.su': 4, 'L2.d': 2, 'L2.w': 4, 'L2.st': 10, 'L2.r': 1, 'L3.su': 2, 'L3.d': 2, 'L3.w': 3, 'L3.st': 12, 'L3.r': 1, 'L4.su': 4, 'L4.d': 2, 'L4.w': 6, 'L4.st': 10, 'L4.r': 0, 'L5.su': 5, 'L5.d': 2, 'L5.w': 2, 'L5.st': 12, 'L5.r': 2, 'L6.su': 1, 'L6.d': 2, 'L6.w': 3, 'L6.st': 10, 'L6.r': 2, 'L7.su': 1, 'L7.d': 2, 'L7.w': 2, 'L7.st': 8, 'L7.r': 2, 'L8.su': 2, 'L8.d': 2, 'L8.w': 3, 'L8.st': 8, 'L8.r': 2, 'L9.su': 5, 'L9.d': 2, 'L9.w': 6, 'L9.st': 8, 'L9.r': 0,```


As you can see, the class 1 has 10 Lessons (from 0 to 9).  
In the first lesson (L0), we assigned the fd - free day - and the fr - favourite room.